# Финальная сборка и заливка табличек с данными о многоквартирных домах, зданиях и сооружениях

Заливаются таблички:
-  <code>apartment_houses_all_data</code>

Перед прогоном тетрадки нужно запустить подготовку данных:
- <code>step1_prepare_dmr_houses_data.ipynb</code>
- <code>step2_prepare_gis_houses_data.ipynb</code>

In [1]:
import json
import pandas as pd
from postamats.utils import prepare_data
from postamats.utils.connections import DB
from postamats.utils.prepare_data import RAW_DMR_NAME, RAW_GIS_NAME, APARTMENT_HOUSES_NAME

pd.set_option('display.max_columns', None)

In [2]:
# путь к json с реквизитами подключения к БД
CONFIG_PATH = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/db_config.json'

In [3]:
with open(CONFIG_PATH, mode='r') as db_file:
    db_config = json.load(db_file)

database = DB(db_config)

In [4]:
prepared_gis = database.get_table_from_bd(RAW_GIS_NAME)
prepared_dmr = database.get_table_from_bd(RAW_DMR_NAME)

Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:115: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:115: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [5]:
prepared_gis.head(2)

,address_gis,address_code_gis,guid_fias_gis,oktmo_code_gis,management_method_gis,management_ogrn_gis,management_kpp_gis,management_name_gis,house_type_gis,condition_gis,total_area_gis,living_area_gis,demolition_date_gis,kad_n_gis,guid_house_gis,object_id_gis
0,"108809, Москва г, п. Марушкинское, д. Марушкин...",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,,,None,Жилой,Исправный,396.2,NaN,None,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09,c03236dfd8117f656eff7f061e9dc22c9034eef194602f...
1,"108809, Москва г, п. Марушкинское, д. Марушкин...",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,,,None,Жилой,Исправный,124.9,NaN,None,нет,407142e2-e27c-4973-b614-7cd939e5a037,26146600ec0d327435e3794dd10b7c7d5df5ebc24b7d53...


In [6]:
prepared_dmr.head(2)

,object_type,on_moscow_territory,address,simple_address,street,local_object_type,local_object_num,korpus_num,stroenie_num,adm_area,district,num_addr_register,date_addr_register,guid_fias,date_fias,kad_n,kad_zu,kladr_code,addr_status,geodata,lat,lon,object_id
0,Здание,да,"Российская Федерация, город Москва, внутригоро...","Косинская улица, дом 26А",Косинская улица,дом,26А,None,None,Восточный административный округ,муниципальный округ Вешняки,3303053,03.08.2004,235212A3-01E8-4CC3-87D5-59F00C83898A,27.02.2012,77:03:0007004:1064,77:03:0007004:6443,77000000000040000,Внесён в ГКН,"{{37.8279504545784,55.7176609928454},{37.82861...",55.717477,37.828208,d2000323020834c81f704195dd68b59499d4800f4b2cae...
1,Здание,да,"Российская Федерация, город Москва, внутригоро...","Гороховский переулок, дом 21",Гороховский переулок,дом,21,None,None,Центральный административный округ,муниципальный округ Басманный,1011856,13.07.2005,533D296D-1ECC-49EA-9156-DCFA8E38E4D8,27.02.2012,77:01:0003010:1018,77:01:0003010:4146,77000000000112200,Внесён в ГКН,"{{37.6682995208392,55.7662431483298},{37.66831...",55.766233,37.668037,5d6cf51cd08bca0baf6680258a2e6a3a293b264b64d6c4...


In [7]:
dmr_fias = set(prepared_dmr['guid_fias'].dropna())
dmr_kadn = set(prepared_dmr['kad_n'].dropna())
dmr_kadzu = set(prepared_dmr['kad_zu'].dropna())

gis_fias = set(prepared_gis['guid_fias_gis'].dropna())
gis_kadn = set(prepared_gis['kad_n_gis'].dropna())

print('gis-dmr fias count:', len(gis_fias), len(dmr_fias))
print('gis-dmr kadn count', len(gis_kadn), len(dmr_kadn))
print('gis-dmr kadn-kadzu count', len(gis_kadn), len(dmr_kadzu))

print('equal fias:', len(gis_fias & dmr_fias))
print('equal kadn:', len(gis_kadn & dmr_kadn))
print('equal kadn-kadzu:', len(gis_kadn & dmr_kadzu))

gis-dmr fias count: 68159 165900
gis-dmr kadn count 47093 126255
gis-dmr kadn-kadzu count 47093 53072
equal fias: 38764
equal kadn: 31245
equal kadn-kadzu: 6


In [8]:
apartment_houses = prepare_data.prepare_apartment_houses_data(prepared_dmr, prepared_gis)

prepare_apartment_houses_data started ... 
prepare_apartment_houses_data finished


In [9]:
apartment_houses

,object_type,on_moscow_territory,address,simple_address,street,local_object_type,local_object_num,korpus_num,stroenie_num,adm_area,district,num_addr_register,date_addr_register,guid_fias,date_fias,kad_n,kad_zu,kladr_code,addr_status,geodata,lat,lon,object_id,address_gis,address_code_gis,guid_fias_gis,oktmo_code_gis,management_method_gis,management_ogrn_gis,management_kpp_gis,management_name_gis,house_type_gis,condition_gis,total_area_gis,living_area_gis,demolition_date_gis,kad_n_gis,guid_house_gis,object_id_gis
0,многоквартирный дом,да,"город Москва, Авиационная улица, дом 65, корпу...","Авиационная улица, дом 65, корпус 3, строение 4",Авиационная улица,дом,65,3,4,Северо-Западный административный округ,муниципальный округ Щукино,8008022,21.09.2006,9E6647F0-DC35-4A33-83DA-24405F5D439C,23.05.2013,77:08:0000000:2781,77:08:0009004:6867,77000000000071200,Внесён в ГКН,"{{37.4539473334854,55.8078787837879},{37.45407...",55.807827,37.453977,c7ad8391b30a404697edcfdea4ba10fa7ea552b4e09269...,"123182, Москва г, ул. Авиационная, д. 65, корп. 3",a00cecf4-a88d-4156-98a1-9157dec5bde1,A00CECF4-A88D-4156-98A1-9157DEC5BDE1,45372000,УО,5137746235611,773401001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,46.8,3597.4,None,77:08:0000000:2781,40f56ef2-aa05-41a5-aade-9da3f090b996,4dfa8a03e8ee627671fbf571901abf381776a9839fd715...
1,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","улица Мусы Джалиля, дом 16, корпус 2, строение 2",улица Мусы Джалиля,дом,16,2,2,Южный административный округ,муниципальный округ Зябликово,5006518,18.11.2003,86BC76E7-A676-4B69-A66D-839A3F37A733,24.11.2011,77:05:0012001:1093,77:05:0012001:1011,77000000000048000,Внесён в ГКН,"{{37.7392150814377,55.6235062851872},{37.73921...",55.623560,37.739392,7b9adf96d39c350de4fdf992f1ff2e497b1f8f5838cd9f...,"115573, Москва г, ул. Мусы Джалиля, д. 16, кор...",275e370b-1719-4150-8727-d85060951270,275E370B-1719-4150-8727-D85060951270,45916000,УО,1157746524231,772401001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,261.0,7225.0,None,77:05:0012001:1093,4fb6a145-0e38-4933-8659-55aa9ddc9522,4eefe3d848079234315dbddcdcc9aa458b3f8f42adfc4f...
2,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","Костромская улица, дом 8А",Костромская улица,дом,8А,None,None,Северо-Восточный административный округ,муниципальный округ Алтуфьевский,2205537,24.11.2004,A4BE6AFA-EE43-47FB-B895-A1B210DC1808,27.02.2012,77:02:0003005:1047,77:02:0003005:43,77000000000155300,Внесён в ГКН,"{{37.5930191098546,55.8857993631666},{37.59301...",55.885641,37.593456,eea53cceda9ab91905922e4fad25b4b146cfb647488f23...,"127549, Москва г, ул. Костромская, д. 6",be80f3a1-810b-414e-802c-236fa1a1a43e,BE80F3A1-810B-414E-802C-236FA1A1A43E,45350000,УО,5147746339362,771501001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,2208.2,10131.3,None,77:02:0003005:1047,8852ff6f-b8d3-4433-8f86-72e6254b1a71,5747c5ed39bc899900b70fac83508f543df84aba0c4990...
3,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","Варшавское шоссе, дом 120, корпус 2",Варшавское шоссе,дом,120,2,None,Южный административный округ,муниципальный округ Чертаново Северное,5015945,22.09.2015,1CC881F1-79D8-47A9-9442-DB7941505175,04.05.2017,77:05:0006004:24499,77:05:0006004:1003,77000000000047600,Внесён в ГКН,"{{37.6173230078642,55.6305115581333},{37.61731...",55.630294,37.617104,9d0f9c193a2b3c86da7b807d19c8884113b0d2421b7155...,"117587, Москва г, ш. Варшавское, д. 120, корп. 2",1b059a8b-53bd-4197-ad33-794d61c3898c,5C284593-7C29-49C9-8EE0-4DEA030D98F9,45924000,УО,1027700082266,773401001,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПИК-...",Многоквартирный,Исправный,13324.9,9005.6,None,77:05:0006004:24499,ed9ceea8-cb2a-4b59-85c1-1647497ed8d3,874706a86f21585085ab35c6a5e6331fc200835f72f1b7...
4,многоквартирный дом,да,"город Москва, Кусковская улица, дом 29, корпус...","Кусковская улица, дом 29, корпус 1, строение 2",Кусковская улиц

In [10]:
database.load_to_bd(apartment_houses, APARTMENT_HOUSES_NAME, geo=False)